## Installation

To get started with the `crawl4ai` integration in AG2, follow these steps:

1. Install AG2 with the `crawl4ai` extra:
   ```bash
   pip install ag2[crawl4ai]
   ```
   > **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
   > ```bash
   > pip install -U autogen[crawl4ai]
   > ```
   > or  
   > ```bash
   > pip install -U pyautogen[crawl4ai]
   > ```
   > as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  
2. Set up Playwright:
   
   ```bash
   # Installs Playwright and browsers for all OS
   playwright install
   # Additional command, mandatory for Linux only
   playwright install-deps
   ```

3. For running the code in Jupyter, use `nest_asyncio` to allow nested event loops.
    ```bash
    pip install nest_asyncio
    ```


You're all set! Now you can start using browsing features in AG2.


## Imports

In [6]:
import os

import nest_asyncio
from pydantic import BaseModel

from autogen.tools.experimental import Crawl4AITool

nest_asyncio.apply()

In [7]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]

llm_config = {
    "config_list": config_list,
}

In [8]:
from autogen.agents.experimental import ReliableFunctionAgent

task = "Which flying unit from 1 tier building in BAR can shoot and stun enemy targets?"

In [9]:
from typing import Annotated

from autogen.tools.dependency_injection import Field as AG2Field

QUESTIONER_PROMPT = """
%s

You are an agent responsible for generating exactly THREE sub_questions that might help answer the user's main question.
In particular:
1. The FIRST and SECOND sub_questions should each explore a different, narrower aspect of the question.
2. The THIRD sub_question should be a broader query that still helps advance understanding of the main question.

You are provided:
1. The main question.
2. A set of facts that currently do not fully answer that question.

Your task:
- Define 3 sub_questions (each sub_question must be answerable with exactly ONE fact).
- Each sub_question must be directly related to the user question and the existing facts.
- No sub_question may reference information or entities not found in the user question or existing facts.
- Each sub_question must be phrased as a single complete question (avoid combining multiple queries with "and").
- Make sure each sub_question is clear and self-contained (no vague pronouns if you already know what or who they refer to).
- The first two sub_questions must focus on two different narrower details or lines of inquiry, while the third is more general or broad.

Finally:
- Invoke the function submit_sub_questions to submit your sub_questions.
"""

QUESTIONER_VALIDATOR_PROMPT = """
%s

You will validate THREE proposed sub_questions for these criteria:

1. **Relevance**: Each sub_question must directly help answer the user's main question or fill a gap in the existing facts.
2. **No External Assumptions**: None of the sub_questions should rely on entities or information not in the user question or existing facts.
3. **Singularity**: Each sub_question must be a single request (no "and" or multiple queries combined).
4. **Clarity & Self-Containment**:
   - No vague pronouns that could cause ambiguity.
   - Each sub_question should stand on its own without depending on context that isn't already explicitly stated.
5. **Variety & Order**:
   - The first two sub_questions must explore different narrower aspects or angles of the main question.
   - The third sub_question must be broader or more open-ended, yet still relevant.

If all three sub_questions meet these conditions, they are valid.
Otherwise, reject or adjust them accordingly.
"""


def submit_sub_questions(
    context_variables: dict,
    sub_questions: Annotated[
        list[str],
        AG2Field(
            description="Based on the facts presented to you, break down the question into a list of specific sub_questions."
        ),
    ],
):
    """
    Use this function to submit sub_questions.
    """
    out = "\n"
    for i, sub_question in enumerate(sub_questions):
        out += f"    {i + 1}: {sub_question}\n"

    return sub_questions, out


questioner = ReliableFunctionAgent(
    name="Questioner",
    runner_llm_config=llm_config,
    validator_llm_config=llm_config,
    func_or_tool=submit_sub_questions,
    runner_system_message=QUESTIONER_PROMPT,
    validator_system_message=QUESTIONER_VALIDATOR_PROMPT,
)

In [10]:
from typing import Annotated

from autogen.tools.dependency_injection import Field as AG2Field

SEARCH_QUERY_GENERATOR_PROMPT = """
%s

You are an agent responsible for generating exactly THREE google_search_queries that might help answer the user's main question.
In particular:
1. The FIRST and SECOND google_search_queries should each explore a different, narrower aspect of the question.
2. The THIRD google_search_queries should be a broader query that still helps advance understanding of the main question.

You are provided:
1. The main question.
2. A set of facts that currently do not fully answer that question.

Your task:
- Define 3 google_search_queries (each google_search_queries must be answerable with exactly ONE fact).
- Each google_search_queries must be directly related to the user question and the existing facts.
- No google_search_queries may reference information or entities not found in the user question or existing facts.
- Each google_search_queries must be phrased as a single complete question (avoid combining multiple queries with "and").
- Make sure each google_search_queries is clear and self-contained (no vague pronouns if you already know what or who they refer to).
- The first two google_search_queries must focus on two different narrower details or lines of inquiry, while the third is more general or broad.

Finally:
- Invoke the function submit_google_search_queries to submit your google_search_queries.
"""

SEARCH_QUERY_GENERATOR_VALIDATOR_PROMPT = """
%s

You will validate THREE proposed google_search_queries for these criteria:

1. **Relevance**: Each sub_question must directly help answer the user's main question or fill a gap in the existing facts.
2. **No External Assumptions**: None of the google_search_queries should rely on entities or information not in the user question or existing facts.
3. **Singularity**: Each sub_question must be a single request (no "and" or multiple queries combined).
4. **Clarity & Self-Containment**:
   - No vague pronouns that could cause ambiguity.
   - Each sub_question should stand on its own without depending on context that isn't already explicitly stated.
5. **Variety & Order**:
   - The first two google_search_queries must explore different narrower aspects or angles of the main question.
   - The third sub_question must be broader or more open-ended, yet still relevant.

If all three google_search_queries meet these conditions, they are valid.
Otherwise, reject or adjust them accordingly.
"""


def submit_google_search_queries(
    context_variables: dict,
    google_search_queries: Annotated[
        list[str],
        AG2Field(
            description="Based on the facts presented to you, break down the question into a list of specific google_search_queries."
        ),
    ],
):
    """
    Use this function to submit google_search_queries.
    """
    out = "\n"
    for i, sub_question in enumerate(google_search_queries):
        out += f"    {i + 1}: {sub_question}\n"

    return google_search_queries, out


search_writer = ReliableFunctionAgent(
    name="SearchWriter",
    runner_llm_config=llm_config,
    validator_llm_config=llm_config,
    func_or_tool=submit_google_search_queries,
    runner_system_message=SEARCH_QUERY_GENERATOR_PROMPT,
    validator_system_message=SEARCH_QUERY_GENERATOR_VALIDATOR_PROMPT,
)

In [ ]:
runner_prompt = """You are a web surfer agent responsible for gathering information from the web for the provided task.
Call crawl4ai with all the provided parameters to execute the scrape."""

validator_prompt = """Validations:
1. Don't validate anything right now, just a placeholder
"""


class SearchResults(BaseModel):
    title: str
    url: str
    description: str


google_searcher = ReliableFunctionAgent(
    name="GoogleSearcher",
    runner_llm_config=llm_config,
    validator_llm_config=llm_config,
    func_or_tool=Crawl4AITool(llm_config=llm_config, extraction_model=SearchResults),
    runner_system_message=runner_prompt,
    validator_system_message=validator_prompt,
)

In [12]:
questions = questioner.run_func(task).result_data
for question in questions:
    search_queries = search_writer.run_func(question).result_data
    for search_query in search_queries:
        search_results = google_searcher.run_func(
            task=f"Do a google search for the query: {search_query}. Collect information from the search results ONLY FOR THE TOP RESULT."
        ).result_data

_User (to chat_manager):

Which flying unit from 1 tier building in BAR can shoot and stun enemy targets?

--------------------------------------------------------------------------------

Next speaker: Questioner-Runner


>>>>>>>> USING AUTO REPLY...
Questioner-Runner (to chat_manager):

***** Suggested tool call (call_TMvV317QNzd1DYfFQXdogx4X): submit_sub_questions *****
Arguments: 
{"sub_questions":["What is the name of the flying unit from a 1 tier building in BAR that can shoot?","What specific ability allows the flying unit to stun enemy targets in BAR?","Are there other units in BAR that can both shoot and stun enemies aside from the flying unit?"],"hypothesis":"There is a flying unit from a 1 tier building in BAR that has the ability to shoot and stun enemy targets."}
*************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Swarm_Tool_Executor


>>>>>>>

KeyboardInterrupt: 